# Imports

In [76]:
# libraries importing
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# additional modules
import sys
sys.path.append('../algorithms')

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from MSET import MSET
from tsad.evaluating.evaluating import evaluating
import numpy as np
import math
from sklearn.metrics import f1_score, fbeta_score

# Load Data

In [79]:
path_to_data = '../data/'

In [80]:
all_files=[]
import os
for root, dirs, files in os.walk(path_to_data):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

In [81]:
# datasets with anomalies loading
list_of_df = [pd.read_csv(file, 
                          sep=';', 
                          index_col='datetime', 
                          parse_dates=True) for file in all_files if 'anomaly-free' not in file]
# anomaly-free df loading
anomaly_free_df = pd.read_csv([file for file in all_files if 'anomaly-free' in file][0], 
                            sep=';', 
                            index_col='datetime', 
                            parse_dates=True)

In [82]:
# dataset characteristics printing
print('SKAB v0.9')
print(f'- A number of datasets: {len(list_of_df)}')
print(f'- Shape of the random dataset: {list_of_df[10].shape}')
n_cp = sum([len(df[df.changepoint==1.]) for df in list_of_df])
n_outlier = sum([len(df[df.anomaly==1.]) for df in list_of_df])
print(f'- A number of changepoints: {n_cp}')
print(f'- A number of outliers (point anomalies): {n_outlier}\n')
print(f'Head of the random dataset:')
display(list_of_df[0].head(1))

SKAB v0.9
- A number of datasets: 34
- Shape of the random dataset: (1146, 10)
- A number of changepoints: 129
- A number of outliers (point anomalies): 13067

Head of the random dataset:


,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly,changepoint
datetime,,,,,,,,,,
2020-03-09 12:14:36,0.027429,0.040353,0.77031,0.382638,71.2129,25.0827,219.789,32.0,0.0,0.0


# Baseline pipeline

In [83]:
def movmean(array, window):

    n = np.size(array)
    xx = array.copy()
    y = []
    for i in range(0, window):
        y.append(np.roll(xx.tolist() + [np.nan]*window, i))
    y = np.nanmean(y, axis=0)
    l = math.ceil(window/2)

    return y[l-1:n+l-1]

In [108]:
predicted_outlier = []
rel_errors = []

In [85]:
predicted_outlier_improved = []
rel_errors_improved = []

In [86]:
def mset_prediction_wrapper(X_array, improved=False):
    """
    This function wraps around the MSET prediction logic to return binary anomaly predictions.
    X_array is expected to be a 2D numpy array where each row is a sample.
    """
    # Ensure the input is a DataFrame with the correct column names
    if not improved:
        X_df = pd.DataFrame(X_array, columns=list_of_df[1].drop(['anomaly','changepoint'], axis=1).columns)
    else:
        X_df = pd.DataFrame(X_array, columns=X_array.columns)
    ms = MSET()
    
    ms.fit(X_df[:400]) 
    # Predict using MSET model
    Y_pred = ms.predict(X_df)
    
    # Calculate errors and relative errors
    err = np.linalg.norm(X_df.values - Y_pred.values, axis=1)
    rel_err = movmean(err / np.linalg.norm(Y_pred.values, axis=1), window=60)
    if improved:
        rel_errors_improved.append(rel_err)
    else:
        rel_errors.append(rel_err)
    
    # Determine binary predictions based on the relative error threshold
    predictions = (rel_err > 0.01).astype(int)
    if improved:
        predicted_outlier_improved.append(pd.DataFrame((rel_err > 0.01), X_df.index).fillna(0).any(axis=1).astype(int))
    else:
        predicted_outlier.append(pd.DataFrame((rel_err > 0.01), X_df.index).fillna(0).any(axis=1).astype(int))
    
    return predictions

# Baseline training

In [109]:
for i in list_of_df:
    mset_prediction_wrapper(i.drop(['anomaly','changepoint'], axis=1))

In [104]:
# true outlier indices selection
true_outlier = [df.anomaly for df in list_of_df]

In [112]:
full_true_outlier = pd.concat(true_outlier)

In [113]:
full_predicted_outlier = pd.concat(predicted_outlier)

In [110]:
# binary classification metrics calculation
binary = evaluating(
    true_outlier, 
    predicted_outlier, 
    metric='binary'
)

Since you not choose numenta_time and portion, then portion will be 0.1
False Alarm Rate 17.93 %
Missing Alarm Rate 16.08 %
F1 metric 0.77


In [114]:
fbeta_score(full_true_outlier.values, full_predicted_outlier.values, beta=2)

0.8111186721500636

# Improvement

In [93]:
from transformers.shap_feature_selector import SHAPFeatureSelector

In [94]:
shap_feature_selector_f_beta = SHAPFeatureSelector(importance=0.85, estimator=mset_prediction_wrapper, beta=2)

In [95]:
for i in list_of_df:
    train_x, train_y = i.drop(['anomaly','changepoint'], axis=1), i['anomaly']
    train_x_imp_beta = shap_feature_selector_f_beta.fit_transform(train_x, train_y.values)
    mset_prediction_wrapper(train_x_imp_beta, improved=True)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [96]:
full_predicted_outlier_improved = pd.concat(predicted_outlier_improved)

# Improved pipeline results

In [97]:
# binary classification metrics calculation
binary = evaluating(
    true_outlier, 
    predicted_outlier_improved, 
    metric='binary'
)

Since you not choose numenta_time and portion, then portion will be 0.1
False Alarm Rate 14.8 %
Missing Alarm Rate 24.7 %
F1 metric 0.74


In [98]:
fbeta_score(full_true_outlier.values, full_predicted_outlier_improved.values, beta=2)

0.748691179156267

# Baseline pipeline results

In [115]:
# binary classification metrics calculation
binary = evaluating(
    true_outlier, 
    predicted_outlier, 
    metric='binary'
)

Since you not choose numenta_time and portion, then portion will be 0.1
False Alarm Rate 17.93 %
Missing Alarm Rate 16.08 %
F1 metric 0.77


In [116]:
fbeta_score(full_true_outlier.values, full_predicted_outlier.values, beta=2)

0.8111186721500636